In [ ]:
import numpy as np
import copy

In [ ]:
def solve_dual_simplex(c: np.array, A: np.array, b: np.array, B: np.array):

    n = len(c)
    m = len(b)

    assert A.shape == (m, n)

    i = 0

    # Main loop
    while True:
        i += 1        
        A_B = A[:, B]
        A_B_inv = np.linalg.inv(A_B)
        c_B = c[B]
        y = c_B @ A_B_inv
        kappa = np.zeros(n)
        kappa[B] = A_B_inv @ b
        if (kappa >= 0).all():
            return kappa
        j_k = np.argmax(kappa < 0)
        k = int(np.where(B == j_k)[0])
        nB = [i for i in range(n) if i not in B]
        miu = {}
        for i in nB:
            miu[i] = A_B_inv[k] @ A[:, i]

        if all([i >= 0 for i in miu.values()]):
            return 'Данная задача несовместна.'
        sigma = {}
        for i in miu.keys():
            if miu[i] < 0: 
                sigma[i] = (c[i] - A[:, i] @ y) / miu[i]
        
        j_0, sigma_0 = list(sigma.items())[0]
        for key, value in sigma.items():
            if value < sigma[j_0]:
                j_0 = key
                sigma_0 = value
                break

        B[k] = j_0

In [3]:
c_hat = np.array([1, 1])
A_hat = np.array([
    [5, 9],
    [9, 5]
])
b_hat = np.array([63, 63])
d_hat_minus = np.array([1, 1])
d_hat_plus = np.array([6, 6])

In [4]:
def solve(c_hat, A_hat, b_hat, d_hat_minus, d_hat_plus):
    m, n = A_hat.shape

    # Шаг 1
    idxs = np.where(c_hat > 0)[0]

    c_hat[idxs] *= -1
    A_hat[:, idxs] *= -1
    d_hat_minus[idxs] *= -1
    d_hat_plus[idxs] *= -1

    d_hat_minus[idxs], d_hat_plus[idxs] = d_hat_plus[idxs], d_hat_minus[idxs]

    # Шаг 2
    alpha = 0
    c = np.hstack((c_hat, np.zeros(m + n)))

    A = np.hstack((
            np.vstack((
                A_hat, 
                np.identity(n)
            )),
            np.identity(m + n)
        ))

    b = np.hstack((b_hat, d_hat_plus))
    d_minus = np.hstack((d_hat_minus, np.zeros(m + n)))
    S = [{
        'alpha': alpha,
        'c': c,
        'A': A,
        'b': b,
        'd_minus': d_minus,
        'delta': copy.deepcopy(d_minus)
    }]
    x_asterisk = None
    r = None

    # Main loop
    while True:
        if len(S) == 0:
            if x_asterisk is None:
                return "Данная задача несовместна."
            
            return -x_asterisk[idxs]
        else:
            last_problem = S.pop()

            alpha = last_problem['alpha']
            c = last_problem['c']
            A = last_problem['A']
            b = last_problem['b']
            d_minus = last_problem['d_minus']
            delta = last_problem['delta']
            alpha_stroke = alpha + c @ d_minus
            b_stroke = b - A @ d_minus
            B = [i for i in range(n, n * 2 + m)]
            x_wave = solve_dual_simplex(c, A, b_stroke, B)
            float_mask = x_wave != x_wave.astype(np.int32)
            if any(float_mask):                
                i = np.argmax(float_mask[:n])
                x_wave_i = x_wave[i]
                if x_asterisk is None or r < np.floor(c @ x_wave + alpha_stroke):
                    b_two_strokes = copy.deepcopy(b_stroke)
                    b_two_strokes[m + i] = np.floor(x_wave_i)
                    S.append({
                        'alpha': copy.deepcopy(alpha_stroke),
                        'c': copy.deepcopy(c),
                        'A': copy.deepcopy(A),
                        'b': b_two_strokes,
                        'd_minus': np.zeros(2 * n + m),
                        'delta': copy.deepcopy(delta),
                    })
                    new_d_minus = np.zeros(2 * n + m)
                    new_d_minus[i] = np.ceil(x_wave_i)
                    S.append({
                        'alpha': copy.deepcopy(alpha_stroke),
                        'c': copy.deepcopy(c),
                        'A': copy.deepcopy(A),
                        'b': copy.deepcopy(b_stroke),
                        'd_minus': new_d_minus,
                        'delta': copy.deepcopy(delta) + new_d_minus,
                    })
            else:
                x_tick = x_wave + delta
                if x_asterisk is None or r < c @ x_tick + alpha_stroke:
                    x_asterisk = x_tick
                    r = c @ x_tick + alpha_stroke

In [ ]:
solve(c_hat, A_hat, b_hat, d_hat_minus, d_hat_plus)

array([4., 4.])